In [ ]:
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS
import os
import scipy.io.wavfile as wavfile

# Initialize an empty dictionary to hold diarization data
dia = {}

# Loop through each file in the 'Raw_Data' folder
for i in os.listdir('Raw_Data'):
    print(f"Processing file: {i}")
    # Perform speaker diarization
    try:
        dia[i[:3]] = aS.speaker_diarization('Raw_Data/' + i, n_speakers=-10)  # Changed aS.speakerDiarization
    except Exception as e:
        print(f"Error during speaker diarization for {i}: {e}")
        continue

    # Mapping speaker numbers every 0.5 seconds
    speaker_mapping = {}
    for idx, speaker in enumerate(dia[i[:3]]):
        speaker_mapping[idx] = speaker

    dia[i[:3]] = speaker_mapping  # Replace with updated dictionary

d:\Major1\venv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Processing file: 308_AUDIO.wav


d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 27763244 bytes, expected 222105644 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
d:\Major1\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing file: 309_AUDIO.wav


d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 22585644 bytes, expected 180684844 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
d:\Major1\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioSegmentation.py:326: RuntimeWarning: invalid value encountered in divide
  transmutation_matrix[i, :] /= transmutation_matrix[i, :].sum()
d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 25139244 bytes, expected 201113644 bytes from header.
  sampling_ra

...

d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
d:\Major1\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing file: 321_AUDIO.wav


d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 26300844 bytes, expected 210406444 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
d:\Major1\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing file: 325_AUDIO.wav


d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 28000044 bytes, expected 224000044 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
d:\Major1\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# Silence Removal and Splitting Based on Speaker
for i in os.listdir('Raw_Data'):
    [Fs, x] = aIO.read_audio_file('Raw_Data/' + i)  
    segments = aS.silence_removal(x, Fs, 0.020, 0.020, smooth_window=1.0, weight=0.3, plot=False) 

    # Check if the audio file is in the diarization dictionary
    if i[:3] not in dia:
        print(f"Diarization data not found for session {i[:3]}, skipping...")
        continue  

    for s in segments:
        try:
            speaker_avg = int((dia[i[:3]][int(Fs * s[0] / 3200)] + dia[i[:3]][int(Fs * s[1] / 3200)]) / 2)
            seg_name = f"{i[:3]}_{s[0]:.2f}-{speaker_avg}.wav"

            output_dir = f"dat_file/{speaker_avg}"
            os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

            wavfile.write(f'{output_dir}/{seg_name}', Fs, x[int(Fs * s[0]):int(Fs * s[1])])

        except KeyError as e:
            print(f"KeyError for segment {s} in session {i[:3]}: {e}")
            os.makedirs('dat_file/others', exist_ok=True)
            seg_name = f"{i[:3]}_{s[0]:.2f}-unknown.wav"
            wavfile.write(f'dat_file/others/{seg_name}', Fs, x[int(Fs * s[0]):int(Fs * s[1])])



d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 27763244 bytes, expected 222105644 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


KeyError for segment [0.68, 1.56] in session 308: 3
KeyError for segment [1.8800000000000001, 9.82] in session 308: 9
KeyError for segment [11.66, 13.200000000000001] in session 308: 58
KeyError for segment [20.080000000000002, 21.34] in session 308: 100
KeyError for segment [24.94, 29.68] in session 308: 124
KeyError for segment [29.76, 30.02] in session 308: 148
KeyError for segment [31.26, 32.18] in session 308: 156
KeyError for segment [33.480000000000004, 36.92] in session 308: 167
KeyError for segment [37.02, 40.46] in session 308: 185
KeyError for segment [40.78, 45.58] in session 308: 203
KeyError for segment [45.86, 55.88] in session 308: 229
KeyError for segment [56.28, 58.06] in session 308: 281
KeyError for segment [59.64, 60.800000000000004] in session 308: 298
KeyError for segment [61.18, 62.02] in session 308: 305
KeyError for segment [62.96, 65.44] in session 308: 314
KeyError for segment [66.16, 67.4] in session 308: 330
KeyError for segment [67.64, 69.76] in session 3

d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 22585644 bytes, expected 180684844 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


Diarization data not found for session 309, skipping...


d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 25139244 bytes, expected 201113644 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


...

KeyError for segment [0.2, 1.48] in session 320: 7
KeyError for segment [2.2800000000000002, 3.86] in session 320: 11
KeyError for segment [4.4, 5.5] in session 320: 22
KeyError for segment [7.42, 8.78] in session 320: 37
KeyError for segment [9.620000000000001, 14.48] in session 320: 48
KeyError for segment [15.66, 17.02] in session 320: 78
KeyError for segment [17.38, 18.32] in session 320: 86
KeyError for segment [20.04, 23.28] in session 320: 100
KeyError for segment [24.48, 24.7] in session 320: 122
KeyError for segment [27.32, 29.16] in session 320: 136
KeyError for segment [29.72, 34.84] in session 320: 148
KeyError for segment [35.5, 40.5] in session 320: 177
KeyError for segment [41.980000000000004, 43.12] in session 320: 209
KeyError for segment [45.92, 47.06] in session 320: 229
KeyError for segment [48.46, 49.42] in session 320: 242
KeyError for segment [51.86, 52.84] in session 320: 259
KeyError for segment [54.54, 55.58] in session 320: 272
KeyError for segment [80.36, 82

d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 26300844 bytes, expected 210406444 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


KeyError for segment [2.1, 2.82] in session 321: 10
KeyError for segment [2.88, 3.86] in session 321: 14
KeyError for segment [5.12, 9.4] in session 321: 25
KeyError for segment [10.3, 11.34] in session 321: 51
KeyError for segment [15.24, 16.22] in session 321: 76
KeyError for segment [18.44, 26.400000000000002] in session 321: 92
KeyError for segment [26.68, 27.78] in session 321: 133
KeyError for segment [59.34, 60.26] in session 321: 296
KeyError for segment [65.06, 66.14] in session 321: 325
KeyError for segment [70.52, 72.3] in session 321: 352
KeyError for segment [73.32000000000001, 74.12] in session 321: 366
KeyError for segment [76.44, 78.2] in session 321: 382
KeyError for segment [81.96000000000001, 83.26] in session 321: 409
KeyError for segment [87.14, 90.42] in session 321: 435
KeyError for segment [94.66, 104.62] in session 321: 473
KeyError for segment [110.16, 118.76] in session 321: 550
KeyError for segment [124.16, 126.04] in session 321: 620
KeyError for segment [1

d:\Major1\venv\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 28000044 bytes, expected 224000044 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


KeyError for segment [5.5, 6.2] in session 325: 27
KeyError for segment [14.84, 15.08] in session 325: 74
KeyError for segment [17.56, 18.34] in session 325: 87
KeyError for segment [54.04, 59.64] in session 325: 270
KeyError for segment [64.38, 73.94] in session 325: 321
KeyError for segment [79.7, 81.06] in session 325: 398
KeyError for segment [82.14, 88.08] in session 325: 410
KeyError for segment [88.44, 93.92] in session 325: 442
KeyError for segment [97.74000000000001, 113.62] in session 325: 488
KeyError for segment [117.84, 121.06] in session 325: 589
KeyError for segment [126.84, 141.68] in session 325: 634
KeyError for segment [148.20000000000002, 154.22] in session 325: 741
KeyError for segment [154.78, 155.96] in session 325: 773
KeyError for segment [156.04, 157.58] in session 325: 780
KeyError for segment [158.0, 161.62] in session 325: 790
KeyError for segment [163.9, 165.16] in session 325: 819
KeyError for segment [165.46, 167.9] in session 325: 827
KeyError for segme

In [ ]:
import pandas as pd
label_df = pd.read_excel("label_data.xlsx")

# Convert it to a dictionary
label_data = dict(zip(label_df["Participant_ID"], label_df["Result"]))

#audio to spectrogram
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import numpy as np
import re

def generate_spectrogram(audio_path, save_path):
    y, sr = librosa.load(audio_path, sr=None)
    plt.figure(figsize=(5, 5))
    D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Usage with your data
for i in os.listdir("dat_file/others/"):
    label = label_data.get(int(i[:3]), "unknown")
    output_dir = f'raw_data_img/{label}'
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'{re.sub(".wav", "", i)}.jpg')
    generate_spectrogram(f'dat_file/others/{i}', output_path)

C:\Users\RIYA SUMRA\AppData\Local\Temp\ipykernel_18268\57059060.py:23: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
C:\Users\RIYA SUMRA\AppData\Local\Temp\ipykernel_18268\57059060.py:23: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
C:\Users\RIYA SUMRA\AppData\Local\Temp\ipykernel_18268\57059060.py:23: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
C:\Users\RIYA SUMRA\AppData\Local\Temp\ipykernel_18268\57059060.py:23: UserWarning: amplitude_to_db was called on comp

In [ ]:
# #random sampling 3000 images of each class
import numpy as np
import cv2
import os

for i in os.listdir('raw_data_img'):
    # Get the files in the current subfolder and select 3000 random ones
    files = os.listdir('raw_data_img/'+i)
    random_files = np.random.choice(files, 3000)
    
    # Create output directory for the current label/category
    output_dir = f'data_img/{i}'
    os.makedirs(output_dir, exist_ok=True)

    nm = 0
    for j in random_files:
        # Read and save the image in the new directory
        im = cv2.imread(f'raw_data_img/{i}/{j}')
        if im is not None:
            cv2.imwrite(f'{output_dir}/{nm}.jpg', im)
            nm += 1


In [5]:
#split folders into test-val
import splitfolders

splitfolders.ratio('data_img', output='final_image_data', seed=1337, ratio=(.8,.20))

Copying files: 6000 files [00:40, 149.28 files/s]
